In [ ]:
!pip install -r requirements.txt

import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

nltk.download('stopwords')
nltk.download('punkt')

from unidecode import unidecode
## ler arquivo json em pandas
df = pd.read_json('project-1-at-2023-05-11-23-47-a0f8a77e.json')

df = df.drop(['id'], axis=1)
## expandir coluna annotations
df = pd.concat([df.drop(['annotations'], axis=1), df['annotations'].apply(pd.Series)], axis=1)
## expandir coluna 0 e renomear para annotations
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna result e renomear para result
df = pd.concat([df.drop(['result'], axis=1), df['result'].apply(pd.Series)], axis=1)
df = df.drop(['id'], axis=1)

## expandir coluna 0 e renomear para result
df = pd.concat([df.drop([0], axis=1), df[0].apply(pd.Series)], axis=1)
## expandir coluna value e renomear para value
df = pd.concat([df.drop(['value'], axis=1), df['value'].apply(pd.Series)], axis=1)
## dropar choices nulos
df = df.dropna(subset=['choices'])
## obter choices 
df['choices'] = df['choices'].apply(lambda x: x[0])

## expandir coluna data
df = pd.concat([df.drop(['data'], axis=1), df['data'].apply(pd.Series)], axis=1)

df_noticia_original = df.copy()


padrao_data_cepea = r"Cepea, \d{2}/\d{2}/\d{4} - "
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_data_cepea, '', x))

## remover a palavra 'cepea' das noticias
padrao_cepea = r"Cepea"
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_cepea, '', x, flags=re.IGNORECASE))

## remover numeros das noticias
padrao_numeros = r'[0-9]+'
df['noticia'] = df['noticia'].apply(lambda x: re.sub(padrao_numeros, '', x))

## noticia que contem a palavra 'soja'
df = df[df['titulo'].str.contains('soja', flags=re.IGNORECASE)]

## remover noticias com choice 'desclassificar'
df = df[df['choices'] != 'Desclassificar']
df.count()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-10-dd8b04155bbc>:33: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['result'], axis=1), df['result'].apply(pd.Series)], axis=1)


file_upload                 421
drafts                      421
predictions                 421
meta                        421
created_at                  421
updated_at                  421
inner_id                    421
total_annotations           421
cancelled_annotations       421
total_predictions           421
comment_count               421
unresolved_comment_count    421
last_comment_updated_at       0
project                     421
updated_by                  421
comment_authors             421
completed_by                421
was_cancelled               421
ground_truth                421
created_at                  421
updated_at                  421
lead_time                   421
prediction                  421
result_count                421
unique_id                   421
last_action                   0
task                        421
project                     421
updated_by                  421
parent_prediction             0
parent_annotation             0
last_cre

In [ ]:
# Selecionar apenas as colunas necessárias
columns_to_select = ['id', 'data', 'noticia', 'titulo', 'choices', 'unique_id']

df = df[columns_to_select]
df.dropna(subset=['noticia'])

# Pré-processamento dos dados
stop_words = set(stopwords.words('portuguese'))

def preprocess_text(text):
    # remover acentuação
    text = unidecode(text)
    # Remover pontuações
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenização
    words = word_tokenize(text.lower())
    # Remover stopwords
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

#df['noticia'] = df['noticia'].apply(preprocess_text)
df_treino = df[:336]
df_validacao = df[336:]


In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
import numpy as np

from typing import List, Optional, Tuple, Union

# Carregar o modelo pré-treinado BERTimbau
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=3)
model = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

modelo_soja = model

modelo_soja

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29794, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class LIABertClassifier(nn.Module):
    def __init__(self,model,num_labels):
        super(LIABertClassifier,self).__init__()
        self.bert = model
        self.config = model.config
        self.num_labels = num_labels
        self.cls = nn.Linear(self.config.hidden_size,200)
        self.dropout = nn.Dropout(p=0.5)
        self.cls2 = nn.Linear(768,num_labels)

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        ) ->Tuple[torch.Tensor]:

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        sequence_output = outputs[0][:,0,:]
        prediction = self.dropout(sequence_output)
        prediction = self.cls2(prediction)
        return prediction
        

In [ ]:
torch.cuda.empty_cache()

from torch.utils.data import DataLoader
from transformers import AdamW, get_scheduler
import sklearn.model_selection as model_selection
from torch.optim.lr_scheduler import OneCycleLR

# Map string labels to integer labels
label_map = {'Negativa': 0, 'Positiva': 1, 'Neutra': 2}
ytrain_global = np.array([label_map[label] for label in df_treino['choices']])
xtrain_global = np.array(df_treino['noticia'])



xtrain, xval, ytrain, yval = model_selection.train_test_split(xtrain_global, ytrain_global, test_size=0.30, random_state=42,shuffle=True)

train_encodings = tokenizer(xtrain.tolist(), truncation=True, padding=True, max_length=512, return_tensors='pt')
val_encodings = tokenizer(xval.tolist(), truncation=True, padding=True,max_length=512, return_tensors='pt')

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        label = torch.tensor(self.labels[idx])
        return (item,label)

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_encodings, ytrain)
val_dataset = MyDataset(val_encodings, yval)

device = torch.device('cuda')

model = LIABertClassifier(modelo_soja, 3)

model.to(device)

dl_train = DataLoader(train_dataset,batch_size=8)
dl_eval  = DataLoader(val_dataset,batch_size=8)
x,y = next(iter(dl_train))

batch = {k: v.to(device) for k, v in x.items()}
model.to(device)

optim = AdamW(model.parameters(), lr=0.000050)

loss_fct = nn.CrossEntropyLoss()

num_epochs = 20

from transformers import get_scheduler

scheduler = get_scheduler(
    "linear",
    optim,
    num_warmup_steps=0,
    num_training_steps=num_epochs*len(dl_train)
)

# scheduler = OneCycleLR(optim, max_lr=max_lr, total_steps=total_steps,
#                        div_factor=div_factor, pct_start=pct_start)

for epoch in range(num_epochs):
    lepochs = []
    for batch,y in dl_train:
        model.train()
        batch = {k: v.to(device) for k, v in batch.items()}
        y = y.to(device)
        outputs = model(**batch)
        loss = loss_fct(outputs,y)
        lepochs.append(loss.cpu().item())
        loss.backward()
        optim.step()
        scheduler.step()
        optim.zero_grad()
        current_lr = optim.param_groups[0]['lr']

        print(f"Epoch {epoch}, Loss={loss:.4f}, LR={current_lr:.6f}")

model.eval()
ytrue = []
ypred = []
for batch,y in dl_eval:
    batch = {k: v.to(device) for k, v in batch.items()}
    y = y.to(device)
    with torch.no_grad():
        outputs = model(**batch)
        ytrue.extend(y.cpu().numpy().tolist())
        ypred.extend(outputs.argmax(axis=1).cpu().numpy().tolist())

print(classification_report(ytrue,ypred))



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0, Loss=1.0939, LR=0.000050
Epoch 0, Loss=0.9335, LR=0.000050
Epoch 0, Loss=1.4008, LR=0.000050
Epoch 0, Loss=1.0125, LR=0.000050
Epoch 0, Loss=1.1969, LR=0.000050
Epoch 0, Loss=1.0570, LR=0.000050
Epoch 0, Loss=0.8592, LR=0.000049
Epoch 0, Loss=1.2487, LR=0.000049
Epoch 0, Loss=0.9386, LR=0.000049
Epoch 0, Loss=1.2180, LR=0.000049
Epoch 0, Loss=1.1786, LR=0.000049
Epoch 0, Loss=1.0961, LR=0.000049
Epoch 0, Loss=1.1410, LR=0.000049
Epoch 0, Loss=1.0289, LR=0.000049
Epoch 0, Loss=1.1304, LR=0.000049
Epoch 0, Loss=1.1338, LR=0.000049
Epoch 0, Loss=1.2214, LR=0.000049
Epoch 0, Loss=0.8935, LR=0.000048
Epoch 0, Loss=1.1160, LR=0.000048
Epoch 0, Loss=1.0662, LR=0.000048
Epoch 0, Loss=1.1953, LR=0.000048
Epoch 0, Loss=1.1424, LR=0.000048
Epoch 0, Loss=1.0490, LR=0.000048
Epoch 0, Loss=1.0104, LR=0.000048
Epoch 0, Loss=1.0263, LR=0.000048
Epoch 0, Loss=1.2263, LR=0.000048
Epoch 0, Loss=1.2370, LR=0.000048
Epoch 0, Loss=1.0619, LR=0.000048
Epoch 0, Loss=1.2064, LR=0.000048
Epoch 0, Loss=

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/modelo_soja.pt')

## predict from string
frase = """
O preço da soja em Paranaguá (PR) em abril registrou o maior aumento médio mensal desde agosto de 2020,
informa o Centro de Estudos Avançados em Economia Aplicada (Cepea), da Esalq/USP.
Considerando a inflação, a média de R$ 145,24 por saca de 60 quilos seria a melhor desde março de 2020.
"""
frase = preprocess_text(frase)
frase = tokenizer(frase, truncation=True, padding=True, max_length=512, return_tensors='pt')
frase = {k: v.to(device) for k, v in frase.items()}
model.eval()
with torch.no_grad():
    outputs = model(**frase)
    if outputs.argmax(axis=1).cpu().numpy().tolist()[0] == 0:
        print('Negativa')
    elif outputs.argmax(axis=1).cpu().numpy().tolist()[0] == 1:
        print('Positiva')
    else:
        print('Neutra')
    print(outputs.argmax(axis=1).cpu().numpy().tolist())



Positiva
[1]
